In [ ]:
# Cell 1: Import everything you need
from vllm_client import (
    VLLMClient,
    MATH_TOOLS, 
    MATH_TOOL_FUNCTIONS, 
    CarDescription,
    CarType
)

# Cell 2: Initialize client
client = VLLMClient()

# Cell 3: Test basic chat
response = client.chat("Tell me a joke")
print(response.choices[0].message.content)

# Cell 4: Test function calling
result = client.chat_with_tools(
    "What is 123 divided by 987?", 
    MATH_TOOLS, 
    MATH_TOOL_FUNCTIONS
)
print(f"Function: {result['tool_call'].name if result['tool_call'] else 'None'}")
print(f"Result: {result['result']}")

# Cell 5: Test structured output
car_schema = CarDescription.model_json_schema()
response = client.structured_chat(
    "Generate a JSON with the brand, model and car_type of the most iconic car from the 90's",
    car_schema
)
print(response.choices[0].message.content)